These are logs from training the following model from the CI script from T4Rec (the trianing was for 5 epochs):

`### XLNet (MLM) - Item Id feature
python3 transf_exp_main_modified.py --output_dir ./tmp/ --overwrite_output_dir --do_train --do_eval --validate_every 10 --logging_steps 20 --save_steps 0 --data_path $DATA_PATH --features_schema_path $FEATURE_SCHEMA_PATH --fp16 --data_loader_engine merlin --start_time_window_index 1 --final_time_window_index 2 --time_window_folder_pad_digits 4 --model_type xlnet --loss_type cross_entropy --per_device_eval_batch_size 128 --similarity_type concat_mlp --tf_out_activation tanh --inp_merge mlp --learning_rate_warmup_steps 0 --learning_rate_schedule linear_with_warmup --hidden_act gelu --num_train_epochs $NUM_EPOCHS --dataloader_drop_last --compute_metrics_each_n_steps 1 --session_seq_length_max 20 --eval_on_last_item_seq_only --mf_constrained_embeddings --layer_norm_featurewise --attn_type bi --mlm --per_device_train_batch_size 128 --learning_rate 0.0006667377132554976 --dropout 0.0 --input_dropout 0.1 --weight_decay 3.910060265627374e-05 --d_model 192 --item_embedding_dim 448 --n_layer 3 --n_head 16 --label_smoothing 0.0 --stochastic_shared_embeddings_replacement_prob 0.1 --item_id_embeddings_init_std 0.11 --other_embeddings_init_std 0.02 --mlm_probability 0.30000000000000004 --eval_on_test_set --seed 100 --report_to none
`

And here are the logs and the results, maybe reproducing that is something that we could work towards (the XLNet with MLM is what I used for benchmarking T4Rec, starting with it would be great)

In [1]:
%%bash

cd /models && git fetch origin && git checkout origin/tf/transformer-api && pip install .
cd /core && git checkout main && git pull origin main && pip install .
cd /nvtabular && git checkout main && git pull origin main && pip install .
cd /systems && git checkout main && git pull origin main && pip install .
cd /dataloader && git checkout main && git pull origin main && pip install .
pip install matplotlib

From https://github.com/NVIDIA-Merlin/Models
 * [new branch]        ci/horovod             -> origin/ci/horovod
 * [new branch]        codespell_fix          -> origin/codespell_fix
   16fb4149..b1c10317  fea-sok-integration-wj -> origin/fea-sok-integration-wj
 * [new branch]        fea-sok-load-dump      -> origin/fea-sok-load-dump
   95462360..a69adf75  gh-pages               -> origin/gh-pages
 * [new branch]        mtl_example            -> origin/mtl_example
   cb431a8a..b90e9a1b  release-22.12          -> origin/release-22.12
 * [new branch]        tf/column_sampling_serialization_fix -> origin/tf/column_sampling_serialization_fix
 * [new branch]        tf/continuous_seq_feats_fix -> origin/tf/continuous_seq_feats_fix
 * [new branch]        tf/dataloader_changes  -> origin/tf/dataloader_changes
 * [new branch]        tf/fix_broadcast_to_sequence -> origin/tf/fix_broadcast_to_sequence
 * [new branch]        tf/fix_training_smaller_accuracy -> origin/tf/fix_training_smaller_accurac

Your branch is behind 'origin/main' by 22 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)


From https://github.com/NVIDIA-Merlin/core
 * branch            main       -> FETCH_HEAD
   5dbafa68..aad0c874 main       -> origin/main


Updating cd96ca5f..aad0c874
Fast-forward
 .github/release-drafter.yml                        |  44 +--
 .github/workflows/ISSUE_TEMPLATE/bug-report.md     |  17 +-
 .../ISSUE_TEMPLATE/documentation-request.md        |  12 +-
 .../workflows/ISSUE_TEMPLATE/feature-request.md    |   5 +-
 .../workflows/ISSUE_TEMPLATE/submit-question.md    |   3 +-
 .github/workflows/ISSUE_TEMPLATE/task.md           |   5 +-
 .github/workflows/cpu-ci.yml                       | 145 +++-------
 .github/workflows/cpu-models.yml                   |  52 ++--
 .github/workflows/cpu-nvtabular.yml                |  52 ++--
 .github/workflows/cpu-packages.yml                 | 126 +++++++++
 .github/workflows/cpu-systems.yml                  |  52 ++--
 .github/workflows/docs-preview-pr.yaml             |   2 +-
 .github/workflows/docs-sched-rebuild.yaml          |   7 +-
 .github/workflows/gpu-ci.yml                       |  30 +-
 .github/workflows/release-drafter.yaml             |   2 +-
 .pre-commit-config.ya

ERROR: Operation cancelled by user


Error while terminating subprocess (pid=1791146): 


In [ ]:
%%bash

rm -rf ecom_dataset
mkdir -p ecom_dataset

pip install gdown
# gdown https://drive.google.com/uc?id=1BvCHc4eXComuNK93bKhRM6cbg9y5p350  # <-- full dataset
gdown https://drive.google.com/uc?id=1NCFZ5ya3zyxPsrmupEoc9UEm4sslAddV
apt-get update -y
apt-get install unzip -y
unzip -d ecom_dataset "rees46_ecom_dataset_small_for_ci.zip"

In [1]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import gc
import numpy as np

import tensorflow as tf

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import merlin.models.tf as mm

/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-09 18:01:08.237320: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 18:01:17.553146: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2023-

In [2]:
# Set the hyperparams similar to T4Rec benchmark script
d_model = 192
n_layer = 3
n_head = 16
batch_size = 128
learning_rate = 0.0006667377132554976
weight_decay = 3.910060265627374e-05 
n_epoch = 5
item_embedding_dim = 448 
item_id_embeddings_init_std = 3
input_dropout = 0.1
initializer_range = 0.02
layer_norm_eps = 1e-12
dropout = 0
mlm_prob = 0.3
eval_on_test_set = True


In [3]:
train = Dataset("ecom_dataset/0001/train.parquet")
if eval_on_test_set:  
    valid = Dataset("ecom_dataset/0002/test.parquet")
else: 
    valid = Dataset("ecom_dataset/0002/valid.parquet")

In [4]:
target = 'sess_pid_seq'

In [5]:
from merlin.models.tf.core.tabular import TabularBlock

# Create equivalent class of T4Rec's TabularDroupout
class TabularDropout(TabularBlock):
    """
    Applies dropout transformation.
    """

    def __init__(self, dropout_rate=0.0):
        super().__init__()
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def forward(self, inputs, **kwargs):
        outputs = {key: self.dropout(val) for key, val in inputs.items()}  # type: ignore
        return outputs

# Create equivalent class of T4Rec's 'layer-norm'
class TabularNorm(TabularBlock):
    """
    Applies layr-norm transformation.
    """

    def __init__(self):
        super().__init__()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def forward(self, inputs, **kwargs):
        outputs = {key: self.layer_norm(val) for key, val in inputs.items()}  # type: ignore
        return outputs



In [6]:
from merlin.schema.io.tensorflow_metadata import TensorflowMetadata

schema = TensorflowMetadata.from_proto_text_file(
    './',
    file_name='rees46_schema_modified.pbtxt'
).to_merlin_schema()

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [7]:
# we only use the item-id as input to the model
schema_model = schema.select_by_tag(Tags.ITEM_ID)

In [8]:
from merlin.models.tf import InputBlockV2
import tensorflow as tf
input_block = InputBlockV2(
    schema_model,
    categorical=mm.Embeddings(
            schema_model.select_by_tag(Tags.CATEGORICAL),
            dim=item_embedding_dim,
            #This is equivalent of torch.nn.init.normal_
            embeddings_initializer=tf.keras.initializers.RandomNormal(
                mean=0.0,
                stddev=item_id_embeddings_init_std
            ),
            sequence_combiner=None,
        ),
    #pre=mm.StochasticSwapNoise(schema_model, replacement_prob=0.1) # This is not working with sequences transforms
    # we apply dropout and layer-norm as post-processing steps before aggregation
    post=TabularDropout(input_dropout).connect(TabularNorm())
    )

In [9]:
# projet the output of the input block into the same dimension as d_model (equivalent of d_output in T4Rec)
mlp_block = mm.MLPBlock(
    [d_model],
    activation='relu',
)

In [10]:
# set the xlnet block with the necessary parameters
xlnet_block = mm.XLNetBlock(
    d_model=d_model, 
    n_head=n_head, 
    n_layer=n_layer, 
    attn_type='bi', 
    hidden_act='gelu', 
    initializer_range=initializer_range, 
    layer_norm_eps=layer_norm_eps, 
    dropout=0
)

In [11]:
dense_block = mm.SequentialBlock(
    input_block,
    mlp_block,
    xlnet_block
)

In [12]:
# Project the output of the transformer to the same dimension as `item_embedding_dim`
# this is needed for weight-tying
mlp_block2 = mm.MLPBlock(
    [item_embedding_dim],
    activation='relu',
)

In [13]:
# set next-item prediction task with weight tying option by providing the embeddings table of the `item-id` 
# as the `to_call` layer
prediction_task = mm.CategoricalOutput(
    to_call=input_block["categorical"][target],
)

In [14]:
# Create the end-to-end Keras model
model_transformer = mm.Model(dense_block, mlp_block2, prediction_task)

2023-03-09 18:01:20.111251: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [15]:
# Implements optimizer with linear decay of the learning rate. This is what T4Rec pytorch-trainer is using. 

# For that we will use the custom optimizer `AdamWeightDecay` provided by HuggingFace
from transformers.optimization_tf import AdamWeightDecay


num_warmup_steps = 0
# compute the total steps in the training iteration:
import math
steps_per_epoch = math.floor(train.compute().shape[0] / batch_size)
total_step = steps_per_epoch * n_epoch

# Set the linear-decay learning scheduler
lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=total_step,
    power=1,
)
# Set the optimizer with the `weight_decay` rate
if weight_decay > 0.0:
    optimizer = AdamWeightDecay(
        learning_rate=lr_schedule,
        weight_decay_rate=weight_decay,
    )
else: 
    optimizer = AdamWeightDecay(learning_rate=lr_schedule)

In [17]:
# compile the model with ranking metrics computed at 10 and 20 thresholds
model_transformer.compile(run_eagerly=False, optimizer=optimizer, loss="categorical_crossentropy",
              metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[10, 20])
             )

In [24]:
# Align the schema of the dataloader and the schema used by the model
train.schema = schema_model

In [19]:
# Launch a training iteration with `n_epoch` epochs
# For mlm, we need to use `SequenceMaskRandom` and specify the masking probability
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequenceMaskRandom(schema=train.schema, target=target, transformer=xlnet_block, masking_prob=mlm_prob)
)

2023-03-09 18:01:47.567695: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/sequential_block_5/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/sequential_block_5/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model/sequential_block_5/xl_net_block/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/sequential_block_5/xl_net_block/sequential_block_8/replace_masked_embe

/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/sequential_block_5/xl_net_block/sequential_block_8/replace_masked_embeddings/RaggedWhere/RaggedTile_2/Reshape_3:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/sequential_block_5/xl_net_block/sequential_block_8/replace_masked_embeddings/RaggedWhere/RaggedTile_2/Reshape_2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model/sequential_block_5/xl_net_block/sequential_block_8/replace_masked_embeddings/RaggedWhere/RaggedTile_2/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:436: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 174720448 elements. This may consume a large amount 

2023-03-09 18:02:45.911807: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model/sequential_block_5/xl_net_block/sequential_block_8/replace_masked_embeddings/RaggedWhere/Assert/AssertGuard/branch_executed/_31


677/677 [==============================] - 202s 204ms/step - loss: 13.5418 - recall_at_10: 0.0333 - mrr_at_10: 0.0120 - ndcg_at_10: 0.0170 - map_at_10: 0.0120 - precision_at_10: 0.0033 - recall_at_20: 0.0524 - mrr_at_20: 0.0133 - ndcg_at_20: 0.0218 - map_at_20: 0.0133 - precision_at_20: 0.0026 - regularization_loss: 0.0000e+00 - loss_batch: 13.5668
Epoch 2/5
677/677 [==============================] - 135s 200ms/step - loss: 12.3608 - recall_at_10: 0.0430 - mrr_at_10: 0.0149 - ndcg_at_10: 0.0214 - map_at_10: 0.0149 - precision_at_10: 0.0043 - recall_at_20: 0.0656 - mrr_at_20: 0.0164 - ndcg_at_20: 0.0271 - map_at_20: 0.0164 - precision_at_20: 0.0033 - regularization_loss: 0.0000e+00 - loss_batch: 12.3602
Epoch 3/5
677/677 [==============================] - 139s 205ms/step - loss: 12.0906 - recall_at_10: 0.0435 - mrr_at_10: 0.0154 - ndcg_at_10: 0.0219 - map_at_10: 0.0154 - precision_at_10: 0.0043 - recall_at_20: 0.0672 - mrr_at_20: 0.0170 - ndcg_at_20: 0.0279 - map_at_20: 0.0170 - precisi

In [25]:
# Evaluate using `SequenceMaskLast` to mask the last item only
valid.schema = schema_model
predict_last = mm.SequenceMaskLast(schema=valid.schema, target=target, transformer=xlnet_block)

In [26]:
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(
2023-03-09 18:28:21.499587: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: model/sequential_block_5/xl_net_block/sequential_block_8/replace_masked_embeddings/RaggedWhere/Assert/AssertGuard/branch_executed/_23


84/84 [==============================] - 34s 128ms/step - loss: 11.7879 - recall_at_10: 0.0458 - mrr_at_10: 0.0155 - ndcg_at_10: 0.0225 - map_at_10: 0.0155 - precision_at_10: 0.0046 - recall_at_20: 0.0711 - mrr_at_20: 0.0171 - ndcg_at_20: 0.0288 - map_at_20: 0.0171 - precision_at_20: 0.0036 - regularization_loss: 0.0000e+00 - loss_batch: 11.7897


{'loss': 11.787938117980957,
 'recall_at_10': 0.0489937923848629,
 'mrr_at_10': 0.017020391300320625,
 'ndcg_at_10': 0.024413621053099632,
 'map_at_10': 0.017020391300320625,
 'precision_at_10': 0.004899379797279835,
 'recall_at_20': 0.07645288854837418,
 'mrr_at_20': 0.018829816952347755,
 'ndcg_at_20': 0.03123626857995987,
 'map_at_20': 0.018829816952347755,
 'precision_at_20': 0.003822644241154194,
 'regularization_loss': 0.0,
 'loss_batch': 11.866037368774414}

In [22]:
{'eval_/next-item/ndcg_at_10': 0.08305524289608002, 'eval_/next-item/ndcg_at_20': 0.09936655312776566, 'eval_/next-item/recall_at_10': 0.15436746180057526, 'eval_/next-item/recall_at_20': 0.2190323770046234, 'eval_/loss': 8.334789276123047}

{'eval_/next-item/ndcg_at_10': 0.08305524289608002,
 'eval_/next-item/ndcg_at_20': 0.09936655312776566,
 'eval_/next-item/recall_at_10': 0.15436746180057526,
 'eval_/next-item/recall_at_20': 0.2190323770046234,
 'eval_/loss': 8.334789276123047}